In [ ]:
#1
import pandas as pd
import xml.etree.ElementTree as ET

df=pd.read_xml('Q2.xml')
print("Original Data Frame:")
print(df)

df['contact'] = df['contact'].str.replace('-', '')
print("\nData Frame after removing '-':")
print(df)

df['name']=df['name'].astype('string').radd('Mr.')
print("\nData Frame after adding 'Mr.' to the names column:")
print(df)


df['contact']=df['contact'].fillna(-1)
df['email_id']=df['email_id'].fillna('UNK')
df['address']=df['address'].fillna('UNK')
df['name']=df['name'].fillna('UNK')
print("\nData Frame after replacing null with 'UNK' for strings and '-1' for numerics:")
print(df)

df.to_csv('output.csv')

In [2]:
#2
import requests
import sqlite3

# Function to fetch earthquake events from USGS API
def fetch_earthquake_events(start_date, end_date):
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
    params = {
        "format": "geojson",
        "starttime": start_date,
        "endtime": end_date,
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch earthquake events:", response.status_code)
        return None

# Function to store earthquake events in SQLite database
def store_events_in_database(events, db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Create table if not exists
    cursor.execute('''CREATE TABLE IF NOT EXISTS earthquakes (
                        id TEXT PRIMARY KEY,
                        time TEXT,
                        latitude REAL,
                        longitude REAL,
                        depth REAL,
                        magnitude REAL,
                        place TEXT
                    )''')

    # Insert each event into the database
    for event in events['features']:
        event_id = event['id']
        event_time = event['properties']['time']
        event_latitude = event['geometry']['coordinates'][1]
        event_longitude = event['geometry']['coordinates'][0]
        event_depth = event['geometry']['coordinates'][2]
        event_magnitude = event['properties']['mag']
        event_place = event['properties']['place']

        cursor.execute('''INSERT OR IGNORE INTO earthquakes
                          (id, time, latitude, longitude, depth, magnitude, place)
                          VALUES (?, ?, ?, ?, ?, ?, ?)''',
                       (event_id, event_time, event_latitude, event_longitude, event_depth, event_magnitude, event_place))

    conn.commit()
    conn.close()

# Main function
def main():
    start_date = "2017-01-01"
    end_date = "2017-12-31"
    db_file = "earthquakes.db"

    # Fetch earthquake events from USGS API
    events = fetch_earthquake_events(start_date, end_date)
    if events:
        # Store events in SQLite database
        store_events_in_database(events, db_file)
        print("Earthquake events stored in the database successfully.")
    else:
        print("Failed to fetch earthquake events.")

if __name__ == "__main__":
    main()


Failed to fetch earthquake events: 400
Failed to fetch earthquake events.
